# Trousbleshooting Open Transaction & Blocking Session


In [ ]:
-- TAKE SCREENSHOT OF THE ENVIRONMENT
SELECT es.session_id, login_name, cpu_time, memory_usage, reads, writes, logical_reads, last_request_start_time, last_request_end_time, last_successful_logon, ib.event_info
FROM sys.dm_exec_sessions AS es
CROSS APPLY sys.dm_exec_input_buffer(es.session_id, NULL) AS ib
WHERE es.session_id > 50
ORDER BY es.last_request_start_time;

In [ ]:
-- CHECK FOR OPEN TRANSACTIONS
SELECT spid ,blocked ,lastwaittype ,db_name(dbid) as'DBName' ,login_time ,last_batch,open_tran ,program_name ,cmd ,loginame ,hostname  ,status  
FROM sys.sysprocesses WHERE open_tran>0 
-- `AND [status] NOT IN ('sleeping','background')


In [ ]:
--CHECK IF THERE IS ANY BLOCKING SESSION(s)
SELECT spid ,blocked ,lastwaittype ,db_name(dbid) as'DBName' ,login_time ,last_batch,open_tran ,program_name ,cmd ,loginame ,hostname  ,status  
FROM sys.sysprocesses WHERE blocked>0

In [ ]:
SELECT w.session_id,w.wait_duration_ms,w.wait_type,w.blocking_session_id,s.[host_name],r.command,r.percent_complete,
       r.cpu_time,r.total_elapsed_time,r.reads,r.writes,r.logical_reads,r.row_count,q.[text],q.dbid,p.query_plan,r.plan_handle
FROM
  sys.dm_os_waiting_tasks w
  INNER JOIN sys.dm_exec_sessions s
    ON w.session_id = s.session_id
  INNER JOIN sys.dm_exec_requests r
    ON s.session_id = r.session_id
  CROSS APPLY sys.dm_exec_sql_text(r.plan_handle) q
  CROSS APPLY sys.dm_exec_query_plan(r.plan_handle) p
WHERE w.session_id > 50--= 
  AND w.wait_type NOT IN
  ('DBMIRROR_DBM_EVENT','ASYNC_NETWORK_IO'
    /* + add your own here*/)

In [ ]:
 -- KILL SESSION(s) CAREFULLY 
--Use this for the sec update
SELECT spid,blocked,lastwaittype,db_name(p.dbid) AS'DBName',login_time,last_batch,open_tran,program_name,cmd,loginame,hostname,loginame,text ,status
FROM sys.sysprocesses  p  
CROSS APPLY sys.dm_exec_sql_text(sql_handle) q  
WHERE blocked>0 OR open_tran>0
  ORDER BY last_batch

In [ ]:
-- KILLING SESSION(s) CAREFULLY
-- KILL @SESSIONID

In [ ]:
create git for bigdatadriven
index column matter